In [34]:
##Old Dominion University Department of Mathematics and Statistics
##Machine Learning II - Prof. Katie Smith
##HW4- Boosting

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier #gradient boosting
import lightgbm as lgb #tree boosting
from sklearn.metrics import roc_auc_score #accuracy metric

In [35]:
train = pd.read_csv('C:/users/emurp010/documents/Kaggle Datasets/Flight Delays/flight_delays_train.csv')
test = pd.read_csv('C:/Users/emurp010/Documents/Kaggle Datasets/Flight Delays/flight_delays_test.csv')

In [36]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [37]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


In [38]:
#getting rid of the hyphen in month, monthday, and weekday columns to make analysis cleaner

train['Month'] = train.Month.apply(lambda x: int(x.split('-')[1]))
train['DayofMonth'] = train.DayofMonth.apply(lambda x: int(x.split('-')[1]))
train['DayOfWeek'] = train.DayOfWeek.apply(lambda x: int(x.split('-')[1])) #use anonymous lambda types to split strings to gather relevant info

carriers_list = train.UniqueCarrier.value_counts().head(10).index.tolist()
train['UniqueCarrier'] = train.UniqueCarrier.apply(lambda x: x if x in carriers_list else 'other') #sorting by distinct carriers

train['dep_delayed_15min'] = train.dep_delayed_15min.apply(lambda x: 1 if x == 'Y' else 0) #encode Y/N binary operator

origin_list = train.Origin.value_counts().head(10).index.tolist()
train['Origin'] = train.Origin.apply(lambda x: x if x in origin_list else 'other') #sorting by distinct origin airports

dest_list = train.Dest.value_counts().head(10).index.tolist()
train['Dest'] = train.Dest.apply(lambda x: x if x in dest_list else 'other') #sorting by distinct destination airports

train = pd.get_dummies(train) #create dummy variables for splitting training set


In [39]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(['dep_delayed_15min'],axis=1), train.dep_delayed_15min, 
                                                    test_size=0.25, random_state=23, stratify=train.dep_delayed_15min)

xgb_model = XGBClassifier(seed=23)

xgb_model.fit(X_train, y_train) #use boosting to fit the training set
xgb_valid_pred = xgb_model.predict_proba(X_test)[:, 1] #use boosting to predict the probability that the independent parameters lead to a delay of >=15min

roc_auc_score(y_test, xgb_valid_pred)

#with xgboost we get to 72.5% with no hyperparameter tuning


0.724777431018381

In [42]:
gbm = lgb.LGBMClassifier(n_estimators=200,)
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        early_stopping_rounds=5) #L1 norm boosting parameter with stopping criterion of less than 5 consecutive iterations

[1]	valid_0's binary_logloss: 0.478262	valid_0's l1: 0.305625
Training until validation scores don't improve for 3 rounds.
[2]	valid_0's binary_logloss: 0.471786	valid_0's l1: 0.303295
[3]	valid_0's binary_logloss: 0.466269	valid_0's l1: 0.301041
[4]	valid_0's binary_logloss: 0.46185	valid_0's l1: 0.29908
[5]	valid_0's binary_logloss: 0.458192	valid_0's l1: 0.297281
[6]	valid_0's binary_logloss: 0.455292	valid_0's l1: 0.295715
[7]	valid_0's binary_logloss: 0.45242	valid_0's l1: 0.294093
[8]	valid_0's binary_logloss: 0.450074	valid_0's l1: 0.292687
[9]	valid_0's binary_logloss: 0.448008	valid_0's l1: 0.291366
[10]	valid_0's binary_logloss: 0.446232	valid_0's l1: 0.290196
[11]	valid_0's binary_logloss: 0.444723	valid_0's l1: 0.289128
[12]	valid_0's binary_logloss: 0.443209	valid_0's l1: 0.288054
[13]	valid_0's binary_logloss: 0.441974	valid_0's l1: 0.287109
[14]	valid_0's binary_logloss: 0.440977	valid_0's l1: 0.286244
[15]	valid_0's binary_logloss: 0.439921	valid_0's l1: 0.285452
[16]	v

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.1, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=200, n_jobs=-1, num_leaves=31, objective=None,
        random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [45]:
gbm.fit(train.drop('dep_delayed_15min', axis=1), train.dep_delayed_15min)
roc_auc_score(y_test, gbm.predict_proba(X_test)[:, 1])

#with no hyperparameter tuning we get to 80.3% fit.

0.8031320251455514